# Amazon Titan 모델을 활용하여 짧은 텍스트 요약

> *This notebook should work well with the **`Data Science 3.0`** kernel in SageMaker Studio*

## Overview

이 예제에서는 소량의 데이터(문자열 데이터)를 Amazon Bedrock API(Amazon Titan 모델 사용)로 직접 수집하고 각 텍스트를 요약하라는 지시를 내릴 것입니다

### Architecture

![](./images/41-text-simple-1.png)

이 아키텍처에서는

1. 작은 텍스트(또는 작은 파일)가 로드됩니다.
1. 기본 모델이 해당 데이터를 처리합니다.
1. 모델이 수집된 텍스트의 요약이 포함된 응답을 반환합니다.

### Use case

이 접근 방식은 통화 녹취록, 회의 녹취록, 책, 기사, 블로그 게시물 및 기타 관련 콘텐츠를 요약하는 데 사용할 수 있습니다.

### Challenges
이 접근 방식은 입력 텍스트나 파일이 모델 컨텍스트 길이에 맞을 때 사용할 수 있습니다. 노트북 `02.long-text-summarization-titan.ipynb`에서는 사용자가 토큰 제한을 초과하는 대용량 문서를 가지고 있을 때 이 문제를 해결하는 접근 방식을 살펴보겠습니다.

## Setup

이 노트북의 나머지 부분을 실행하기 전에 아래 셀을 실행하여 필요한 라이브러리가 설치되어 있는지 확인하고 베드락에 연결해야 합니다.

설정 작동 방식과 ⚠️ **변경이 필요한지 여부**에 대한 자세한 내용은 [Bedrock 기본환경 설정 노트북](../00_Setup/setup.ipynb) 노트북을 참조하세요.

In [ ]:
import os
import sys
import json
import boto3

module_path = ".."
sys.path.append(os.path.abspath(module_path))
from utils import bedrock, print_ww

In [ ]:
is_internal_use = True # True: AWS, False: Client
#bedrock_region = "" ## <your region> 
bedrock_region = "us-west-2"

In [ ]:
if bedrock_region == "us-east-1":
    bedrock_config = {
        "region_name":bedrock_region,
        "endpoint_url": "https://bedrock.us-east-1.amazonaws.com" if is_internal_use else None
    }
elif bedrock_region == "us-west-2":
    bedrock_config = {
        "region_name":bedrock_region,
        "endpoint_url": "https://prod.us-west-2.frontend.bedrock.aws.dev" if is_internal_use else None
    }

In [ ]:
if is_internal_use:
    boto3_bedrock = boto3.client(
        service_name='bedrock',
        region_name=bedrock_config["region_name"],
        endpoint_url=bedrock_config["endpoint_url"]
    )
else:
    boto3_bedrock = boto3.client(
        service_name='bedrock',
        region_name=bedrock_config["region_name"]
    ) 

In [ ]:
%pip install --quiet langchain==0.0.249

## Summarizing a short text with boto3
 
To learn detail of API request to Amazon Bedrock, this notebook introduces how to create API request and send the request via Boto3 rather than relying on langchain, which gives simpler API by wrapping Boto3 operation. 

### boto3로 짧은 텍스트 요약하기


파운데이션 모델에 요청을 보내기 위해 `InvokeModel` API를 사용합니다. 다음은 Amazon Titan Text Large에 텍스트를 전송하기 위한 API 요청 예제입니다. `textGenerationConfig`의 추론 파라미터는 사용하려는 모델에 따라 다릅니다. Amazon Titan Text Large의 추론 파라미터는 다음과 같습니다:
- **maxTokenCount****는 생성된 응답에 사용할 최대 토큰 수를 구성합니다. (int, 기본값은 512)
- **stopSequences**는 문장이나 목록의 끝과 같이 원하는 지점에서 모델을 멈추게 하는 데 사용됩니다. 반환된 응답에는 중지 시퀀스가 포함되지 않습니다.
- **temperature**는 온도 샘플링 기법을 구현하여 다음 토큰에 대한 확률 밀도 함수를 조절합니다. 이 매개변수는 밀도 함수 곡선을 깊게 또는 평평하게 만드는 데 사용할 수 있습니다. 값이 낮을수록 곡선이 가파르고 결정론적 응답이 많아지는 반면, 값이 높을수록 곡선이 평평해지고 무작위 응답이 많아집니다. (플로트, 기본값은 0, 최대값은 1.5)
- **Top P**는 잠재적 선택의 확률에 따라 토큰 선택을 제어합니다. Top P를 1.0 미만으로 설정하면 모델은 가장 가능성이 높은 옵션만 고려하고 가능성이 낮은 옵션은 무시합니다. 그 결과 더 안정적이고 반복적인 완료가 가능합니다.

```python
response = bedrock.invoke_model(body={
                                   "inputText": "this is where you place your input text",
                                   "textGenerationConfig": {
                                       "maxTokenCount": 4096,
                                       "stopSequences": [],
                                       "temperature":0,
                                       "topP":1
                                       },
                                },
                                modelId="amazon.titan-tg1-large", 
                                accept=accept, 
                                contentType=contentType)

```

### 요약할 텍스트가 있는 쓰기 프롬프트

이 노트북에서는 토큰이 기초 모델의 최대 토큰보다 작은 짧은 텍스트를 사용할 수 있습니다. 짧은 텍스트의 예로 아마존 베드락 발표에 대한 [AWS 블로그 포스트](https://aws.amazon.com/jp/blogs/machine-learning/announcing-new-tools-for-building-with-generative-ai-on-aws/)의 한 단락을 예로 들어 보겠습니다.

프롬프트는 `Please provide a summary of the following text.`라는 명령어로 시작되며, `<text>` 태그로 둘러싸인 텍스트가 포함되어 있습니다.

In [ ]:
prompt = """
Please provide a summary of the following text. Do not add any information that is not mentioned in the text below.

<text>
AWS took all of that feedback from customers, and today we are excited to announce Amazon Bedrock, \
a new service that makes FMs from AI21 Labs, Anthropic, Stability AI, and Amazon accessible via an API. \
Bedrock is the easiest way for customers to build and scale generative AI-based applications using FMs, \
democratizing access for all builders. Bedrock will offer the ability to access a range of powerful FMs \
for text and images—including Amazons Titan FMs, which consist of two new LLMs we’re also announcing \
today—through a scalable, reliable, and secure AWS managed service. With Bedrock’s serverless experience, \
customers can easily find the right model for what they’re trying to get done, get started quickly, privately \
customize FMs with their own data, and easily integrate and deploy them into their applications using the AWS \
tools and capabilities they are familiar with, without having to manage any infrastructure (including integrations \
with Amazon SageMaker ML features like Experiments to test different models and Pipelines to manage their FMs at scale).
</text>

"""

## 프롬프트 및 추론 매개 변수를 사용하여 요청 본문 만들기

`invoke_model`의 요청 구문에 따라 위의 프롬프트와 추론 매개변수를 사용하여 요청 본문을 생성합니다.

In [ ]:
body = json.dumps({"inputText": prompt, 
                   "textGenerationConfig":{
                       "maxTokenCount":4096,
                       "stopSequences":[],
                       "temperature":0,
                       "topP":1
                   },
                  }) 

## Boto3를 통해 파운데이션 모델 호출

요청 매개변수 `modelId`, `accept`, `contentType`을 지정하여 Amazon Bedrock에 API 요청을 전송합니다. 프롬프트에 따라 Amazon Bedrock의 기초 모델이 텍스트를 합산합니다.

In [ ]:
modelId = 'amazon.titan-tg1-large' # change this to use a different version from the model provider
accept = 'application/json'
contentType = 'application/json'

response = boto3_bedrock.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
response_body = json.loads(response.get('body').read())

print_ww(response_body.get('results')[0].get('outputText'))

위의 예에서 Bedrock 서비스는 주어진 프롬프트에 대한 전체 요약을 단일 출력으로 생성하지만, 출력에 많은 양의 토큰이 포함된 경우 속도가 느려질 수 있습니다. 

아래에서는 사용자가 모델에서 생성되는 즉시 사용할 수 있도록 베드락을 사용하여 출력을 스트리밍하는 방법을 살펴봅니다. 이를 위해 베드락은 청크 형태로 출력을 스트리밍하는 `ResponseStream`을 제공하는 `invoke_model_with_response_stream` API를 지원합니다.

In [ ]:
response = boto3_bedrock.invoke_model_with_response_stream(body=body, modelId=modelId, accept=accept, contentType=contentType)
stream = response.get('body')
output = list(stream)
output

베드락은 한번에 전체 출력을 생성하는 대신 작은 청크(Chunk)단위로 나누어 생성합니다.

In [ ]:
from IPython.display import display_markdown, Markdown, clear_output

In [ ]:
response = boto3_bedrock.invoke_model_with_response_stream(body=body, modelId=modelId, accept=accept, contentType=contentType)
stream = response.get('body')
output = []
i = 1
if stream:
    for event in stream:
        chunk = event.get('chunk')
        if chunk:
            chunk_obj = json.loads(chunk.get('bytes').decode())
            text = chunk_obj['outputText']
            clear_output(wait=True)
            output.append(text)
            display_markdown(Markdown(''.join(output)))
            i+=1

clear_output(wait=True)
print_ww(''.join(output))

## Conclusion
이제 아마존 베드락 API에 대한 바닐라 노출을 제공하는 `boto3` SDK를 사용하여 실험해 보았습니다. 이 API를 사용하여 전체 출력과 스트리밍 출력 생성의 두 가지 방법으로 Amazon Bedrock에 대한 AWS 뉴스 요약을 생성하는 사용 사례를 살펴보았습니다.

### Take aways
- 이 노트북을 개조하여 Anthropic Claude 및 AI21 Labs Jurassic 모델과 같은 Amazon Bedrock을 통해 제공되는 다양한 모델을 실험해 보세요.
- 프롬프트를 특정 사용 사례에 맞게 변경하고 다양한 모델의 출력을 평가하세요.
- 토큰 길이를 변경하여 서비스의 지연 시간과 응답성을 파악하세요.
- 다양한 프롬프트 엔지니어링 원칙을 적용하여 더 나은 결과를 얻으세요.

## Thank You